In [3]:
import requests
import pandas as pd
from tqdm import tqdm

In [4]:
# async with httpx.AsyncClient() as client:
#     overall_data = await client.get('https://fantasy.premierleague.com/api/bootstrap-static/')
overall_data = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
overall_data = overall_data.json()

In [5]:
df = pd.DataFrame(overall_data['elements'])

In [6]:
df.index=df['id']

In [7]:
columns_to_remove = ['first_name','photo','news','news_added','second_name','squad_number','web_name','code']

In [8]:
df.drop(columns_to_remove,axis=1,inplace=True)
len(df.columns)

80

In [9]:
df = df[(df.status != "u")]
len(df)

617

In [10]:
train = []
test = []
for i in tqdm(range(len(df['id']))):
    player_data = requests.get('https://fantasy.premierleague.com/api/element-summary/'+str(df['id'].iloc[i])+'/')
    player_data = player_data.json()
    for i in player_data['history']:
        train.append(i)
    for i in player_data['fixtures']:
        test.append(i)

100%|██████████| 617/617 [08:35<00:00,  1.20it/s]


In [11]:
train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

In [18]:
train_df.to_excel("train.xlsx",sheet_name="train_data")

In [19]:
test_df.to_excel("test.xlsx",sheet_name="test_data")

In [4]:
train_df = pd.read_excel('train.xlsx')
test_df = pd.read_excel('test.xlsx')

In [5]:
train_df.columns

Index(['Unnamed: 0', 'element', 'fixture', 'opponent_team', 'total_points',
       'was_home', 'kickoff_time', 'team_h_score', 'team_a_score', 'round',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out'],
      dtype='object')

In [19]:
test_df

,Unnamed: 0,id,code,team_h,team_h_score,team_a,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty
0,0,201,2367739,1,NaN,8,NaN,21.0,False,0,False,2024-01-20T12:30:00Z,Gameweek 21,True,2
1,1,215,2367752,16,NaN,1,NaN,22.0,False,0,False,2024-01-30T19:30:00Z,Gameweek 22,False,2
2,2,221,2367759,1,NaN,11,NaN,23.0,False,0,False,2024-02-04T16:30:00Z,Gameweek 23,True,4
3,3,239,2367776,19,NaN,1,NaN,24.0,False,0,False,2024-02-11T14:00:00Z,Gameweek 24,False,3
4,4,242,2367779,6,NaN,1,NaN,25.0,False,0,False,2024-02-17T15:00:00Z,Gameweek 25,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,11157,340,2367877,20,NaN,1,NaN,34.0,False,0,False,2024-04-20T14:00:00Z,Gameweek 34,True,4
11158,11158,350,2367887,20,NaN,12,NaN,35.0,False,0,False,2024-04-27T14:00:00Z,Gameweek 35,True,2
11159,11159,359,2367896,13,NaN,20,NaN,36.0,False,0,False,2024-05-04T14:00:00Z,Gameweek 36,False,5
11160,11160,370,2367907,20,NaN,8,NaN,37.0,False,0,False,2024-05-11T14:00:00Z,Gameweek 37,True,2


In [41]:
train_x = train_df[train_df['element']==355]
train_x = train_x[['round','expected_goal_involvements','opponent_team','minutes']]
# train_x

In [23]:
import numpy as np

In [42]:
test_x = test_df[train_df['element']==355]
test_x['Opponent_team'] = np.where(test_x['is_home'] > True, test_x['team_h'], test_x['team_a'])
test_x = test_x[['Opponent_team']]
test_x

C:\Users\Dr. B. N. Sivasankar\AppData\Local\Temp\ipykernel_10048\4143203310.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_x = test_df[train_df['element']==355]
C:\Users\Dr. B. N. Sivasankar\AppData\Local\Temp\ipykernel_10048\4143203310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['Opponent_team'] = np.where(test_x['is_home'] > True, test_x['team_h'], test_x['team_a'])


,Opponent_team
7431,15
7432,6
7433,14
7434,1
7435,14
7436,18
7437,14
7438,19
7439,14
7440,14


In [43]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(endog = train_x['expected_goal_involvements'], exog=train_x[['opponent_team']] , order=(1, 1, 1))


c:\Users\Dr. B. N. Sivasankar\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Dr. B. N. Sivasankar\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Dr. B. N. Sivasankar\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [44]:
model_fit = model.fit()
forecast = model_fit.forecast(steps=10, exog=test_x['Opponent_team'].iloc[:10])

c:\Users\Dr. B. N. Sivasankar\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Dr. B. N. Sivasankar\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [45]:
forecast

20   -0.036948
21   -0.242319
22   -0.053541
23   -0.374414
24   -0.050234
25    0.054385
26   -0.048983
27    0.078959
28   -0.048511
29   -0.048047
Name: predicted_mean, dtype: float64

In [46]:
train_x['expected_goal_involvements']

7431    0.65
7432    0.45
7433    1.94
7434    1.68
7435    2.59
7436    0.70
7437    0.09
7438    0.17
7439    0.26
7440    2.51
7441    0.46
7442    2.06
7443    0.74
7444    0.99
7445    0.83
7446    0.00
7447    0.00
7448    0.00
7449    0.00
7450    0.00
Name: expected_goal_involvements, dtype: float64

In [39]:
train_df.columns

Index(['Unnamed: 0', 'element', 'fixture', 'opponent_team', 'total_points',
       'was_home', 'kickoff_time', 'team_h_score', 'team_a_score', 'round',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out'],
      dtype='object')

In [40]:
test_df.columns

Index(['Unnamed: 0', 'id', 'code', 'team_h', 'team_h_score', 'team_a',
       'team_a_score', 'event', 'finished', 'minutes',
       'provisional_start_time', 'kickoff_time', 'event_name', 'is_home',
       'difficulty'],
      dtype='object')